In [1]:
import pandas as pd
import numpy as np

In [2]:
!ls

predictions_plain.csv  result_b5_t15_combined.csv  sample_data


In [3]:
# predictions_plain.csv (unknown number of prediction columns)
pred = pd.read_csv("predictions_plain.csv")
pred.columns

Index(['filename', 'GTX_TITAN_Black', 'RTX_2080_Ti', 'TITAN_V', 'GTX_TITAN_X',
       'GTX_4070'],
      dtype='object')

In [4]:
pred.head()

,filename,GTX_TITAN_Black,RTX_2080_Ti,TITAN_V,GTX_TITAN_X,GTX_4070
0,10003_2_deltaCalcOutput.cu,2.5,2.5,3.5,2.5,3.5
1,10008_81_softmax_gradient_kernel.cu,3.2,2.5,2.5,2.5,2.5
2,10003_2_deltaCalcOutput.cu,2.5,3.5,3.5,2.1,2.5
3,10008_81_softmax_gradient_kernel.cu,3.5,3.5,3.5,2.5,2.5
4,1000_24_transposedMatrixKernel.cu,2.5,2.0,2.0,2.0,0.8


In [5]:
# predictions_plain.csv (unknown number of prediction columns)
true = pd.read_csv("result_b5_t15_combined.csv")
true.columns

Index(['filename', 'kernel_name', 'GeForce GTX TITAN Black (us)',
       'GeForce RTX 2080 Ti (us)', 'TITAN V (us)', 'GeForce GTX TITAN X(us)',
       'GeForce GTX 4070'],
      dtype='object')

In [6]:
true.head()

,filename,kernel_name,GeForce GTX TITAN Black (us),GeForce RTX 2080 Ti (us),TITAN V (us),GeForce GTX TITAN X(us),GeForce GTX 4070
0,10003_2_deltaCalcOutput.cu,10003_2_deltaCalcOutput,0.075397,2.32554,10.2958,4.86060,8.00380
1,10008_81_softmax_gradient_kernel.cu,10008_81_softmax_gradient_kernel,0.070320,4.21498,17.4469,10.89450,8.52477
2,1000_24_transposedMatrixKernel.cu,1000_24_transposedMatrixKernel,0.067752,3.71225,10.2010,6.61657,8.23261
3,1000_25_additionMatricesKernel.cu,1000_25_additionMatricesKernel,0.068978,5.25263,14.1516,9.68920,8.19156
4,1000_26_additionMatricesKernel.cu,1000_26_additionMatricesKernel,0.068879,5.26246,15.2192,9.59985,8.29490


In [7]:
df = pd.merge(true, pred, on="filename", how="inner")

In [8]:
df.head()

,filename,kernel_name,GeForce GTX TITAN Black (us),GeForce RTX 2080 Ti (us),TITAN V (us),GeForce GTX TITAN X(us),GeForce GTX 4070,GTX_TITAN_Black,RTX_2080_Ti,TITAN_V,GTX_TITAN_X,GTX_4070
0,10003_2_deltaCalcOutput.cu,10003_2_deltaCalcOutput,0.075397,2.32554,10.2958,4.86060,8.00380,2.5,2.5,3.5,2.5,3.5
1,10003_2_deltaCalcOutput.cu,10003_2_deltaCalcOutput,0.075397,2.32554,10.2958,4.86060,8.00380,2.5,3.5,3.5,2.1,2.5
2,10008_81_softmax_gradient_kernel.cu,10008_81_softmax_gradient_kernel,0.070320,4.21498,17.4469,10.89450,8.52477,3.2,2.5,2.5,2.5,2.5
3,10008_81_softmax_gradient_kernel.cu,10008_81_softmax_gradient_kernel,0.070320,4.21498,17.4469,10.89450,8.52477,3.5,3.5,3.5,2.5,2.5
4,1000_24_transposedMatrixKernel.cu,1000_24_transposedMatrixKernel,0.067752,3.71225,10.2010,6.61657,8.23261,2.5,2.0,2.0,2.0,0.8


In [11]:
df.columns[3], df.columns[8]

('GeForce RTX 2080 Ti (us)', 'RTX_2080_Ti')

In [14]:
import numpy as np
import pandas as pd

gpu_pairs = [
    ("GeForce GTX TITAN Black (us)", "GTX_TITAN_Black"),
    ("GeForce RTX 2080 Ti (us)", "RTX_2080_Ti"),
    ("TITAN V (us)", "TITAN_V"),
    ("GeForce GTX TITAN X(us)", "GTX_TITAN_X"),
    ("GeForce GTX 4070", "GTX_4070"),
]

df_errors = pd.DataFrame()
df_errors["filename"] = df["filename"]
df_errors["kernel_name"] = df["kernel_name"]

for true_col, pred_col in gpu_pairs:

    true_vals = df[true_col].astype(float)
    pred_vals = df[pred_col].astype(float)

    AE = (pred_vals - true_vals).abs()
    SE = (pred_vals - true_vals) ** 2
    PE = AE / np.clip(true_vals, 1e-12, None) * 100

    df_errors[f"{pred_col}_AE"] = AE
    df_errors[f"{pred_col}_SE"] = SE
    df_errors[f"{pred_col}_PE_%"] = PE

df_errors.head()


,filename,kernel_name,GTX_TITAN_Black_AE,GTX_TITAN_Black_SE,GTX_TITAN_Black_PE_%,RTX_2080_Ti_AE,RTX_2080_Ti_SE,RTX_2080_Ti_PE_%,TITAN_V_AE,TITAN_V_SE,TITAN_V_PE_%,GTX_TITAN_X_AE,GTX_TITAN_X_SE,GTX_TITAN_X_PE_%,GTX_4070_AE,GTX_4070_SE,GTX_4070_PE_%
0,10003_2_deltaCalcOutput.cu,10003_2_deltaCalcOutput,2.424603,5.878700,3215.781795,0.17446,0.030436,7.501914,6.7958,46.182898,66.005556,2.36060,5.572432,48.566021,4.50380,20.284214,56.270771
1,10003_2_deltaCalcOutput.cu,10003_2_deltaCalcOutput,2.424603,5.878700,3215.781795,1.17446,1.379356,50.502679,6.7958,46.182898,66.005556,2.76060,7.620912,56.795457,5.50380,30.291814,68.764837
2,10008_81_softmax_gradient_kernel.cu,10008_81_softmax_gradient_kernel,3.129680,9.794897,4450.625711,1.71498,2.941156,40.687738,14.9469,223.409820,85.670807,8.39450,70.467630,77.052641,6.02477,36.297854,70.673696
3,10008_81_softmax_gradient_kernel.cu,10008_81_softmax_gradient_kernel,3.429680,11.762705,4877.246871,0.71498,0.511196,16.962833,13.9469,194.516020,79.939130,8.39450,70.467630,77.052641,6.02477,36.297854,70.673696
4,1000_24_transposedMatrixKernel.cu,1000_24_transposedMatrixKernel,2.432248,5.915830,3589.927973,1.71225,2.931800,46.124318,8.2010,67.256401,80.394079,4.61657,21.312719,69.772858,7.43261,55.243691,90.282547


In [15]:
avg_error_data = {}

for _, pred_col in gpu_pairs:
    pe_col = f"{pred_col}_PE_%"
    avg_error_data[pred_col] = df_errors[pe_col].mean()

df_gpu_avg_error = pd.DataFrame.from_dict(avg_error_data, orient="index", columns=["Avg_PE_%"])

print("\nAverage percentage error per GPU:")
df_gpu_avg_error


Average percentage error per GPU:


,Avg_PE_%
GTX_TITAN_Black,12775.944034
RTX_2080_Ti,95.162064
TITAN_V,142.826793
GTX_TITAN_X,144.710238
GTX_4070,80.435600


In [16]:
# List of filenames to select
selected_kernels = [
    "1_0_euclidean_kernel.cu",
    "1_2_maximum_kernel.cu",
    "1_14_getColNorms.cu",
    "1_15_gpuFindMax.cu",
    "1_16_gpuSwapCol.cu",
    "1_17_makeHVector.cu",
    "1_18_UpdateHHNorms.cu",
    "1_19_gpuKendall.cu",
    "1_20_getRestricted.cu",
    "1_21_getUnrestricted.cu",
    "1_22_ftest.cu",
    "1_36_gpuMeans.cu",
    "1_37_gpuSD.cu"
]

# Create new dataframe with only these rows
df_selected = df_errors[df_errors["filename"].isin(selected_kernels)].reset_index(drop=True)

df_selected


,filename,kernel_name,GTX_TITAN_Black_AE,GTX_TITAN_Black_SE,GTX_TITAN_Black_PE_%,RTX_2080_Ti_AE,RTX_2080_Ti_SE,RTX_2080_Ti_PE_%,TITAN_V_AE,TITAN_V_SE,TITAN_V_PE_%,GTX_TITAN_X_AE,GTX_TITAN_X_SE,GTX_TITAN_X_PE_%,GTX_4070_AE,GTX_4070_SE,GTX_4070_PE_%
0,1_0_euclidean_kernel.cu,1_0_euclidean_kernel,5.108531,26.097089,5584.986170,1.79384,3.217862e+00,42.773210,1.80101,3.243637e+00,31.602266,0.08398,7.052640e-03,1.132413,0.82858,6.865448e-01,9.948635
1,1_14_getColNorms.cu,1_14_getColNorms,2.221698,4.935942,2837.345151,46.70380,2.181245e+03,93.028416,36.94420,1.364874e+03,83.124907,171.89500,2.954789e+04,95.819281,17.62440,3.106195e+02,83.431482
2,1_15_gpuFindMax.cu,1_15_gpuFindMax,3.428785,11.756567,4814.694938,3.88845,1.512004e+01,60.866877,1.63253,2.665154e+00,24.613986,194.13800,3.768956e+04,98.229085,5.15346,2.655815e+01,67.335035
3,1_16_gpuSwapCol.cu,1_16_gpuSwapCol,2.427122,5.890921,3330.390516,0.55472,3.077143e-01,21.713534,1.30746,1.709452e+00,34.339428,4.45772,1.987127e+01,64.068689,5.83223,3.401491e+01,69.996027
4,1_17_makeHVector.cu,1_17_makeHVector,7.431124,55.221604,10789.134096,17.65600,3.117343e+02,88.474644,23.44090,5.494758e+02,75.760240,57.76770,3.337107e+03,88.644681,11.51090,1.325008e+02,76.683610
5,1_18_UpdateHHNorms.cu,1_18_UpdateHHNorms,1.931205,3.729553,2807.188022,0.34635,1.199583e-01,14.761225,3.76737,1.419308e+01,80.717192,3.01351,9.081243e+00,54.656834,6.20829,3.854286e+01,75.634389
6,1_19_gpuKendall.cu,1_19_gpuKendall,11.223373,125.964101,14646.760280,5.91793,3.502190e+01,374.062462,9.63135,9.276290e+01,577.194139,0.72319,5.230038e-01,43.128917,6.58694,4.338778e+01,142.788951
7,1_20_getRestricted.cu,1_20_getRestricted,81.897783,6707.246860,80121.489576,8906.05000,7.931773e+07,99.874401,16886.40000,2.851505e+08,99.955605,12911.20000,1.666991e+08,99.913329,6133.24000,3.761663e+07,99.959255
8,1_21_getUnrestricted.cu,1_21_getUnrestricted,27.303449,745.478327,28278.784269,801.93900,6.431062e+05,98.622622,1103.78000,1.218330e+06,93.084721,3285.75000,1.079615e+07,98.353663,567.36900,3.219076e+05,98.695355
9,1_22_ftest.cu,1_22_ftest,2.193920,4.813285,2068.174962,100.01000,1.000200e+04,97.561214,94.57050,8.943579e+03,94.788039,194.91300,3.799108e+04,98.733619,67.43970,4.548113e+03,96.425492


In [17]:
avg_error_data = {}

for _, pred_col in gpu_pairs:
    pe_col = f"{pred_col}_PE_%"
    avg_error_data[pred_col] = df_selected[pe_col].mean()

df_gpu_avg_selected = pd.DataFrame.from_dict(avg_error_data, orient="index", columns=["Avg_PE_%"])

print("\nAverage percentage error per GPU (selected kernels):")
df_gpu_avg_selected



Average percentage error per GPU (selected kernels):


,Avg_PE_%
GTX_TITAN_Black,13075.851830
RTX_2080_Ti,87.250531
TITAN_V,105.565851
GTX_TITAN_X,74.024158
GTX_4070,79.027875


In [19]:
import pandas as pd

# Data you provided
gpu_names = [
    "GTX TITAN Black",
    "GeForce RTX 2080 Ti",
    "TITAN V",
    "GeForce GTX TITAN X",
    "GeForce GTX 4070"
]

avg_errors = [
    25947.00908,
    97.91098358,
    74.4615631,
    89.56441338,
    61.99633077
]

# Create dataframe
df_avg_manual = pd.DataFrame({
    "GPU": gpu_names,
    "Avg_PE_%": avg_errors
})

df_avg_manual


,GPU,Avg_PE_%
0,GTX TITAN Black,25947.009080
1,GeForce RTX 2080 Ti,97.910984
2,TITAN V,74.461563
3,GeForce GTX TITAN X,89.564413
4,GeForce GTX 4070,61.996331
